In [7]:
import pandas as pd
import tensorflow as tf
import datatable as dt
import re
import string
import nltk
from nltk.stem import 	WordNetLemmatizer

In [14]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mahboub\AppData\Roaming\nltk_data...


True

In [ ]:
gpus = tf.config.list_physical_devices("GPU")
if gpus:
     for gpu in gpus:
         print("Found a GPU with the name:", gpu)
else:
     print("Failed to detect a GPU.")

In [32]:
df= pd.read_csv('sentiment140.csv')

In [5]:
with tf.device('/GPU:0'):
    count =0
    for i in df['sentiment']:
        if i==4:
            df.loc[count,'sentiment']=1
        count+=1

In [3]:
data=dt.fread('sentiment140.csv')

In [33]:
df.loc[df['sentiment']==4,'sentiment']=1

In [34]:
def change_url_mention_hashtag_punct(text):
    # replace urls
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    text_without_urls = url_pattern.sub("URL", text)
    # replace hashtag
    hashtagh_pattern = re.compile(r"#\S+")
    text_without_urls_hashtag=hashtagh_pattern.sub("HASHTAG",text_without_urls)
     # replace mention
    mention_pattern = re.compile(r'@\S+')
    text_without_urls_hashtag_mention = mention_pattern.sub("MENTION", text_without_urls_hashtag)
    #remove punctuation
    text=text.translate(str.maketrans('', '', string.punctuation))
    return text_without_urls_hashtag_mention
df['text']=df['text'].apply(change_url_mention_hashtag_punct)

In [35]:
df['text'] = df['text'].apply(nltk.word_tokenize)

In [36]:
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatizing_list_of_token(tokens):
    result=[]
    for i in tokens:
        result.append(wordnet_lemmatizer.lemmatize(i))
    return result

# df['text']=df['text'].apply(wordnet_lemmatizer.lemmatize)

df['text']= df['text'].apply(lemmatizing_list_of_token)

In [38]:
df['text'].head(20)

0     [MENTION, URL, -, Awww, ,, that, 's, a, bummer...
1     [is, upset, that, he, ca, n't, update, his, Fa...
2     [MENTION, I, dived, many, time, for, the, ball...
3     [my, whole, body, feel, itchy, and, like, it, ...
4     [MENTION, no, ,, it, 's, not, behaving, at, al...
5                      [MENTION, not, the, whole, crew]
6                                        [Need, a, hug]
7     [MENTION, hey, long, time, no, see, !, Yes, .....
8             [MENTION, nope, they, did, n't, have, it]
9                          [MENTION, que, me, muera, ?]
10    [spring, break, in, plain, city, ..., it, 's, ...
11                       [I, just, re-pierced, my, ear]
12    [MENTION, I, could, n't, bear, to, watch, it, ...
13    [MENTION, It, it, count, ,, idk, why, I, did, ...
14    [MENTION, i, would, 've, been, the, first, ,, ...
15    [MENTION, I, wish, I, got, to, watch, it, with...
16    [Hollis, ', death, scene, will, hurt, me, seve...
17                               [about, to, fil

In [41]:
df.to_csv('preProccessedDataSet.csv',index=False)

In [43]:

# Shuffle dataframe using sample function
df = df.sample(frac=1)
ratio = 0.8
 
total_rows = df.shape[0]
train_size = int(total_rows*ratio)
 
# Split data into test and train
train = df[0:train_size]
test = df[train_size:]


In [44]:
train.to_csv('train.csv',index=False)
test.to_csv('test.csv',index=False)